# Casestudy: Sudokusolver 

In dieser Casestudy wollen wir mit Hilfe von Cython einen Sudokusolver beschleunigen.

## Python 🐌

Das hier ist die Ausgangsimplementierung, welche wir letztlich bestmöglich beschleunigen möchten:

In [ ]:
def solver_py(puzzle):
    row, col = find_empty_cell(puzzle)
    if row == -1 or col == -1:
        return True

    for num in range(1,10):
        if valid(puzzle, num, (row, col)):
            puzzle[row, col] = num
            if solver_py(puzzle):
                return True
            else:
                puzzle[row, col] = 0
    return False

def find_empty_cell(puzzle):
    for i in range(puzzle.shape[0]):
        for j in range(puzzle.shape[0]):
            if puzzle[i, j] == 0:
                return (i, j)
    return (-1, -1)

def valid(puzzle, num, pos):
    # Check row and column
    for k in range(puzzle.shape[0]):
        if puzzle[pos[0], k] == num or puzzle[k, pos[1]] == num:
            return False
    # Check Box
    bx, by = 3*(pos[1] // 3), 3*(pos[0] // 3)
    for i in range(by, by+3):
        for j in range(bx, bx+3):
            if puzzle[i, j] == num:
                return False
    return True

In [ ]:
import numpy as np
S = np.loadtxt("worlds_hardest.txt", dtype=np.int32)

In [ ]:
def solve_py(S):
    puzzle = S.copy()
    if solver(puzzle):
        return puzzle



**Aufgabe**: Time den obigen Pythoncode bzw. die Funktion `solvepy`.



**Aufgabe:** *Profile* den obigen Pythoncode mit Hilfe des line_profilers um die laufzeitkritischen Stellen des Codes zu finden. Profile lediglich die Funktion `solver_py` bzw. den Aufruf `solver_py(S.copy())`.

In [ ]:
%load_ext line_profiler

## Python + Numpy

Als erste Lösung schreiben wir den obigen Code mit Hilfe von numpy wie folgt um:

In [ ]:
def solver_numpy(puzzle):
    row, col = find_empty_cell_numpy(puzzle)
    if row == -1 or col == -1:
        return True

    for num in range(1,10):
        if valid_numpy(puzzle, num, (row, col)):
            puzzle[row, col] = num
            if solver_numpy(puzzle):
                return True
            else:
                puzzle[row, col] = 0
    return False

def find_empty_cell_numpy(puzzle):
    empty_pos = np.argwhere(puzzle == 0)
    if empty_pos.size > 0:
        return empty_pos[0]
    return (-1, -1)

def valid_numpy(puzzle, num, pos):
    # Check row and column
    if np.where((puzzle[pos[0], :] == num) | (puzzle[:, pos[1]] == num))[0].size >= 1:
        return False
    # Check box
    bx, by = 3*(pos[1] // 3), 3*(pos[0] // 3)
    if np.where(puzzle[by:by+3, bx:bx+3] == num)[0].size >= 1:
        return False
    return True

In [ ]:
def solve_numpy(S):
    puzzle = S.copy()
    if solver_numpy(puzzle):
        return puzzle

**Aufgabe**: Time und profile diese Implementierung. Beim profilen analog zu oben `solver_numpy` bzw. `solver_numpy(S.copy())` profilen. Warum hält sich der Speedup im Vergleich zur Pythonimplementierung in Grenzen?

## Cython 🚀

**Aufgabe:** 

- Beschleunige den ursprünglichen Pythoncode (nicht die numpy Implementierung) bestmöglich mit Cython (ohne threadbasierte Parallelisierung). 
- Kompiliere den Code stets mit `-a` um Stellen mit Pythonoverhead zu erkennen. 
- Verwende eine neue Funktion `solve_cy`.
- Time deinen Code letztlich erneut und vergleiche den Speedup zu den bisherigen Implementierungen.

In [ ]:
%load_ext cython

# Endgegner

**Aufgabe:** Nachdem wir unseren Solver hoffentlich ordentlich mit Cython beschleunigt haben, wollen wir ihn jetzt anhand eines **richtig** schwierigen Sudokus testen. Time deine `solve_cy` Funktion und schätze anhand des Speedups ab, wie lange die ursprüngliche Implementierung `solve_py` für dieses Sudoku benötigt hätte.

In [ ]:
Shard = np.loadtxt("backtracking_hard.txt", dtype=np.int32)